# Пошук власних чисел матриць

In [13]:
import numpy as np
from sympy import *

In [14]:
n = 40
m = 7
l = 0
k = 4
i = 6
j = 0

A = np.array([[n, 7, 2, 4, m, -3],
              [5, -l, -2, -3, -3, 2],
              [2, -1, k, -1, 2, -2*j],
              [2*l, 5, -1, -i, 2, -2],
              [-2, -4, 3, 1, j, -2],
              [-1, -3*k, 8, 2, -2, m]])

In [15]:
A

array([[ 40,   7,   2,   4,   7,  -3],
       [  5,   0,  -2,  -3,  -3,   2],
       [  2,  -1,   4,  -1,   2,   0],
       [  0,   5,  -1,  -6,   2,  -2],
       [ -2,  -4,   3,   1,   0,  -2],
       [ -1, -12,   8,   2,  -2,   7]])

# Метод Фаддєєва-Леверье

In [16]:
def faddeev_leverye_method(matrix):
    K = np.diag(np.ones(matrix.shape[0]))
    b = [1]
    for i in np.arange(matrix.shape[0]):
        b.append(-(1/(i+1))*np.trace(np.matmul(matrix, K)))
        K = np.matmul(A, K) + b[i + 1]*np.diag(np.ones(matrix.shape[0]))
    return np.array(b)

def build_polynomial(coefficients):
    x = symbols("x")
    polynomial = 0
    for i in np.arange(coefficients.size):
        polynomial += x**(coefficients.size - i - 1)*coefficients[i]
    return polynomial

In [17]:
b = faddeev_leverye_method(A)
polynomial = build_polynomial(b)

In [18]:
polynomial

1.0*x**6 - 45.0*x**5 + 150.0*x**4 + 1116.0*x**3 - 7484.0*x**2 - 51900.0*x + 36288.0

# Метод Крилова

In [19]:
def krylovs_method(matrix, vector):
    M = []
    M.append(vector)
    for i in np.arange(matrix.shape[0] - 1):
        M.append(np.dot(np.linalg.matrix_power(matrix, i+1), vector))
    M = np.array(M)
    M = M.transpose()
    M = np.flip(M, axis=1)
    if np.linalg.det(M) == 0:
        print("Оберіть інший вектор")
        exit()
    x_n = np.dot(np.linalg.matrix_power(matrix, matrix.shape[0]), vector)
    return np.array(np.concatenate(([1], np.linalg.solve(M, -x_n)), axis=None))

In [20]:
b = krylovs_method(A, [1, 0, 0, 0, 0, 0])
polynomial = build_polynomial(b)

In [21]:
polynomial

1.0*x**6 - 45.000000000005*x**5 + 150.000000000232*x**4 + 1115.99999999893*x**3 - 7484.00000000379*x**2 - 51899.9999999454*x + 36288.0000001959

# Корені характеристичного полінома

In [22]:
np.roots(b)

array([40.77938309+0.j        ,  6.11066969+4.44428675j,
        6.11066969-4.44428675j, -4.32302314+2.33760399j,
       -4.32302314-2.33760399j,  0.64532381+0.j        ])

In [23]:
np.linalg.eigvals(A)

array([40.77938309+0.j        ,  6.11066969+4.44428675j,
        6.11066969-4.44428675j,  0.64532381+0.j        ,
       -4.32302314+2.33760399j, -4.32302314-2.33760399j])

# Ітераційний метод на основі QR декомпозиції

In [24]:
def qr_algorithm(matrix, number_of_iterations):
    for i in np.arange(number_of_iterations):
        Q, R = np.linalg.qr(matrix)
        matrix = np.matmul(R, Q)
    print(matrix)
    eigvals = []
    j = 0
    while j < matrix.shape[0]:
        if j+1 < matrix.shape[0] and matrix[j+1][j] != 0:
            i = symbols("i")
            b = matrix[j][j] + matrix[j+1][j+1]
            D = abs(b**2 - 4*(matrix[j][j]*matrix[j+1][j+1] 
                    - matrix[j][j+1]*matrix[j+1][j]))
            eigvals.append(b/2 + (sqrt(D)/2)*i)
            eigvals.append(b/2 - (sqrt(D)/2)*i)
            j += 2
        else:
            eigvals.append(matrix[j][j])
            j += 1
    return np.array(eigvals)

In [25]:
qr_algorithm(A, 10**5)

[[40.77938309  4.21957293 -0.1583263   4.36406043  7.74259928  1.25064059]
 [ 0.          3.26840558  3.78463175  0.09271825  4.15368926 -2.03405446]
 [ 0.         -7.35346311  8.95293379 12.19786521  2.84598216  1.4334282 ]
 [ 0.          0.          0.         -5.2969855  -1.97317301  3.07693862]
 [ 0.          0.          0.          3.25009265 -3.34906078 -3.07645908]
 [ 0.          0.          0.          0.          0.          0.64532381]]


array([40.77938308995223, 4.44428674560195*i + 6.11066968730882,
       6.11066968730882 - 4.44428674560195*i,
       2.33760399314684*i - 4.32302313976035,
       -2.33760399314684*i - 4.32302313976035, 0.6453238149514212],
      dtype=object)